# Deep Learning - Assignment 3

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

## Process Data

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1 - L2 Regularizaton

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
##################################################
# SGD with 1-hidden layer n.n. with RUL's and L2 #
##################################################

batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
    biases1 = tf.Variable(tf.zeros([hidden1_units]))
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    # Set final layer variables
    weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden1, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    ## validation operation
    valid_prediction1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction1, weights) + biases)

    #test operation
    test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [6]:
from time import time
start = time()

num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print("Initialized")
    print('')
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print('At step %d ' % step)
            print('-----------')
            print("Minibatch loss %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print('')
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

end = time()
print('')
print('Spend seconds: %d' % (end - start))

Initialized

At step 0 
-----------
Minibatch loss 823.064087
Minibatch accuracy: 9.4%
Validation accuracy: 25.8%

At step 500 
-----------
Minibatch loss 227.338348
Minibatch accuracy: 82.8%
Validation accuracy: 78.7%

At step 1000 
-----------
Minibatch loss 103.818794
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%

At step 1500 
-----------
Minibatch loss 48.614143
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%

At step 2000 
-----------
Minibatch loss 23.152588
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%

At step 2500 
-----------
Minibatch loss 11.205336
Minibatch accuracy: 84.4%
Validation accuracy: 87.7%

At step 3000 
-----------
Minibatch loss 5.647856
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%

Test accuracy: 93.5%

Spend seconds: 12


##  Problem 2 - Restrict your training data

Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
##################################################
# SGD with 1-hidden layer n.n. with RUL's and L2 #
##################################################
# Few batches #
###############

batch_size = 5
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
    biases1 = tf.Variable(tf.zeros([hidden1_units]))
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    # Set final layer variables
    weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden1, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    ## validation operation
    valid_prediction1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction1, weights) + biases)

    #test operation
    test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [8]:
from time import time
start = time()

num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print("Initialized")
    print('')
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('At step %d ' % step)
            print('-----------')
            print("Minibatch loss %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print('')
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

end = time()
print('')
print('Spend seconds: %d' % (end - start))

Initialized

At step 0 
-----------
Minibatch loss 766.146790
Minibatch accuracy: 0.0%
Validation accuracy: 14.4%

At step 500 
-----------
Minibatch loss 2717493761560777180747661312.000000
Minibatch accuracy: 20.0%
Validation accuracy: 27.7%

At step 1000 
-----------
Minibatch loss nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%

At step 1500 
-----------
Minibatch loss nan
Minibatch accuracy: 20.0%
Validation accuracy: 10.0%

At step 2000 
-----------
Minibatch loss nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%

At step 2500 
-----------
Minibatch loss nan
Minibatch accuracy: 40.0%
Validation accuracy: 10.0%

At step 3000 
-----------
Minibatch loss nan
Minibatch accuracy: 20.0%
Validation accuracy: 10.0%

Test accuracy: 10.0%

Spend seconds: 8


## Problem 3 - Dropout

Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
##################################################
# SGD with 1-hidden layer n.n. with RUL's and L2 #
##################################################
# Dropout #
###########

batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer 1
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1_units]))
    biases1 = tf.Variable(tf.zeros([hidden1_units]))
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden1=tf.nn.dropout(hidden1, 0.5)

    # Set final layer variables
    weights = tf.Variable(tf.truncated_normal([hidden1_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden1, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    ## validation operation
    valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction1, weights) + biases)

    test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [14]:
from time import time
start = time()

num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print("Initialized")
    print('')
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('At step %d ' % step)
            print('-----------')
            print("Minibatch loss %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print('')
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

end = time()
print('')
print('Spend seconds: %d' % (end - start))

Initialized

At step 0 
-----------
Minibatch loss 948.596985
Minibatch accuracy: 11.7%
Validation accuracy: 26.3%

At step 500 
-----------
Minibatch loss 232.800064
Minibatch accuracy: 77.3%
Validation accuracy: 79.0%

At step 1000 
-----------
Minibatch loss 107.441811
Minibatch accuracy: 73.4%
Validation accuracy: 81.9%

At step 1500 
-----------
Minibatch loss 49.076801
Minibatch accuracy: 78.9%
Validation accuracy: 84.0%

At step 2000 
-----------
Minibatch loss 23.401136
Minibatch accuracy: 80.5%
Validation accuracy: 85.7%

At step 2500 
-----------
Minibatch loss 11.350611
Minibatch accuracy: 78.9%
Validation accuracy: 86.3%

At step 3000 
-----------
Minibatch loss 5.746343
Minibatch accuracy: 79.7%
Validation accuracy: 86.6%

Test accuracy: 92.3%

Spend seconds: 12


## Problem 4 - Try to get the best performance

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
---

In [15]:
import math as math

batch_size = 128
initial_learning_rate_value = 0.5
beta1 = 0.001
beta2 = 0.001
beta3 = 0.001
beta4 = 0.001
beta5 = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)    

    # learning rate decay
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 100, 0.95)

    # Hidden layer 1
    h1_size = 1024  
    weights_h1 = tf.Variable(tf.truncated_normal([image_size * image_size, h1_size], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_h1 = tf.Variable(tf.zeros([h1_size]))
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
    h1 =tf.nn.dropout(h1, 0.5)

    # Hidden layer 2
    h2_size = 1024
    weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
    biases_h2 = tf.Variable(tf.zeros([h2_size]))
    h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)
    h2 =tf.nn.dropout(h2, 0.5)

    # Hidden layer 3
    h3_size = 305
    weights_h3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
    biases_h3 = tf.Variable(tf.zeros([h3_size]))
    h3 = tf.nn.relu(tf.matmul(h2, weights_h3) + biases_h3)
    h3 =tf.nn.dropout(h3, 0.5)

    # Hidden layer 4
    h4_size = 75
    weights_h4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
    biases_h4 = tf.Variable(tf.zeros([h4_size]))
    h4 = tf.nn.relu(tf.matmul(h3, weights_h4) + biases_h4)
    h4 =tf.nn.dropout(h4, 0.5)

    # Output layer
    weights_o = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
    biases_o = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(h4, weights_o) + biases_o
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) + (beta3 * tf.nn.l2_loss(weights_h3)) + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
    train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
    train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
    train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
    train_logits = tf.matmul(train_h4, weights_o) + biases_o
    train_prediction = tf.nn.softmax(train_logits)

    valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
    valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
    valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
    valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
    valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
    valid_prediction = tf.nn.softmax(valid_logits)

    test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
    test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
    test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
    test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
    test_logits = tf.matmul(test_h4, weights_o) + biases_o
    test_prediction = tf.nn.softmax(test_logits)

In [16]:
from time import time
start = time()

num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print("Initialized")
    print('')
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('At step %d ' % step)
            print('-----------')
            print("Minibatch loss %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print('')
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

end = time()
print('')
print('Spend seconds: %d' % (end - start))

Initialized

At step 0 
-----------
Minibatch loss 5.009369
Minibatch accuracy: 10.2%
Validation accuracy: 15.1%

At step 500 
-----------
Minibatch loss 1.968390
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%

At step 1000 
-----------
Minibatch loss 1.729009
Minibatch accuracy: 79.7%
Validation accuracy: 85.8%

At step 1500 
-----------
Minibatch loss 1.272528
Minibatch accuracy: 86.7%
Validation accuracy: 86.4%

At step 2000 
-----------
Minibatch loss 1.156162
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%

At step 2500 
-----------
Minibatch loss 1.203414
Minibatch accuracy: 84.4%
Validation accuracy: 87.5%

At step 3000 
-----------
Minibatch loss 1.073765
Minibatch accuracy: 85.2%
Validation accuracy: 87.8%

Test accuracy: 93.3%

Spend seconds: 30
